In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [22]:
scada_data = pd.read_csv("C:\\Users\\hanna\\Desktop\\data\\ScadaData.txt", delimiter='\t',  parse_dates=True)
scada_data['dtTimeStamp'] = pd.to_datetime(scada_data['dtTimeStamp'])
scada_data.index = scada_data['dtTimeStamp']

single_sensor_wind_speed = scada_data[scada_data["intObjectId"] == 1748][["WindSpeedMean"]]

### Rolling statistics
If the rolling statistics exhibit a clear trend (upwards or downwards) and show varying variance (increasing or decreasing amplitude), then you might conclude that the series is very likely not to be stationary.

#### Rolling Mean

In [23]:
fig, ax = plt.subplots(1,figsize=(12, 9))
ax.plot(scada_data.index, scada_data["WindSpeedMean"], label='raw data')
ax.plot(scada_data["WindSpeedMean"].rolling(window=12).mean(), label="rolling mean");
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Rolling Variance (Standard Deviation)

In [26]:
fig, ax = plt.subplots(1,figsize=(12, 9))
ax.plot(scada_data.index, scada_data["WindSpeedMean"], label='raw data')
ax.plot(scada_data["WindSpeedMean"].rolling(window=12).std(), label="rolling std");
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Augmented Dickey-Fuller Test

#### KPSS

#### Autocorrelation plots (ACF & PACF)